In [ ]:
#Overarching importation
import sys, os
PROJECT_ROOT = os.path.abspath("..")  # go up one directory from notebooks/
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

In [ ]:
from src.Data_ingestor import DataIngestorFactory
from src.Missing_value_handling import MissingValueHandler,FillMissingValue,DropMissingValue

#-------#
# To help with reading and manipulating data
import pandas as pd
import numpy as np

# To help with data visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# To be used for missing value imputation
from sklearn.impute import SimpleImputer

# To help with model building
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

# To get different metric scores, and split data
from sklearn import metrics
#---#
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score
)

# To oversample and undersample data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# To be used for data scaling and one hot encoding
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# To be used for tuning the model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from lightgbm import LGBMClassifier

# To use statistical functions
import scipy.stats as stats

# To be used for creating pipelines and personalizing them
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To supress warnings
import warnings

warnings.filterwarnings("ignore")

In [ ]:
#Load Data
BASE_URL = "http://127.0.0.1:8000"
TRAIN_ENDPOINT = f"{BASE_URL}/train"
TEST_ENDPOINT = f"{BASE_URL}/test"

In [ ]:
#Load Data
BASE_URL = "http://127.0.0.1:8000"
TRAIN_ENDPOINT = f"{BASE_URL}/train"
TEST_ENDPOINT = f"{BASE_URL}/test"
df_train = DataIngestorFactory.get_data_ingestor(TRAIN_ENDPOINT).ingest(TRAIN_ENDPOINT)
df_test = DataIngestorFactory.get_data_ingestor(TEST_ENDPOINT).ingest(TEST_ENDPOINT)

In [ ]:
handler = MissingValueHandler(FillMissingValue,method = 'mean')

handler.handle_missing_values(df_train)

In [ ]:
from src.Data_Samplier import SamplerFactory

samplier = SamplerFactory.create('smoteenn')
df_scaled = samplier.impute(df_train,target_col= 'Target')

In [ ]:
#If we want to scale up Trainig Set
from src.Data_Samplier import SMOTEENNSampler
df_scaled = SMOTEENNSampler().impute(df_train,target_col='Target')
df_scaled['Target'].value_counts()

In [ ]:
from src.Model_Selector import CrossValidationEvaluation,ModelEvaluator

In [ ]:
X = df_scaled.drop('Target', axis=1)
y = df_scaled['Target']

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2,     
    random_state=42,    
    stratify=y           
)


In [ ]:
# #Model Selection
models = [
    ("log", LogisticRegression(solver="newton-cg", random_state=42)),
    ("DecisionTreeClassifier", DecisionTreeClassifier(random_state=42)),
    ("XGBClassifier", XGBClassifier(random_state=42, eval_metric="logloss", device='cpu'))
]

In [ ]:
# Step 1: create evaluation strategy
strategy = CrossValidationEvaluation(n_splits=5, random_state=42)

# Step 2: create the evaluator with that strategy
evaluator = ModelEvaluator(strategy=strategy)

# Step 3: evaluate  models
results = evaluator.evaluate_models(models=models, X=X_train, y=y_train)

# Step 4: Return best model name and best model attribute
best_model_name,best_model = evaluator.get_best_model(models)


In [ ]:
#Model Tunning
from src.Model_Tuner import OptunaTuning

#Step 1 Define Tuning configuration
tuner = OptunaTuning(config_dir="config", n_trials=50, cv_folds=5)

#Step 2 Run Tuning to get final model, parameter and best score
best_model, best_params, best_score = tuner.tune("xgbclassifier", XGBClassifier, X_train, y_train)


In [ ]:
tuner.save_tuned_model("xgbclassifier", best_model)

In [ ]:
from src.Model_Evaluator import ClassificationModelEvaluator

In [ ]:
eval_map = {
        "classification": '123'
       # "regression": 
}

In [1]:
BASE_URL = "http://127.0.0.1:8000"
TRAIN_ENDPOINT = f"{BASE_URL}/train"
TEST_ENDPOINT = f"{BASE_URL}/test"

In [2]:
#Overarching importation
import sys, os
PROJECT_ROOT = os.path.abspath("..")  # go up one directory from notebooks/
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)
from zenml import pipeline
from steps.Data_Ingestion_Steps import data_ingestion_step
from steps.Data_Split_Steps import data_split_step
from steps.Data_Sampling_Steps import data_sampling_step
from steps.Model_Selection_Steps import model_selection_step
from steps.Model_Tuning_Steps import model_tuning_step
from steps.Missing_data_Handling_Steps import missing_value_step
import os
import sys
import time
import traceback
from datetime import datetime
from src.Get_Logging_Config import get_logger
import joblib

In [3]:
logger = get_logger(__name__)

In [4]:
#Data Ingestion Step
logger.info("Starting data ingestion")
raw_data = data_ingestion_step(TRAIN_ENDPOINT)
logger.info(f"Data ingestion completed: {raw_data.shape}")

2025-10-22 21:26:06 [INFO] __main__: Starting data ingestion
Starting data ingestion
Running single step pipeline to execute step data_ingestion_step
Initiating a new run for the pipeline: data_ingestion_step.
In a future release, the default Python package installer used by ZenML to build container images for your containerized pipelines will change from 'pip' to 'uv'. To maintain current behavior, you can explicitly set python_package_installer=PythonPackageInstaller.PIP in your DockerSettings.
Caching is disabled by default for data_ingestion_step.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step data_ingestion_step has started.
[data_ingestion_step] 2025-10-22 21:26:09 [INFO] src.Data_ingestor: Fetching data from API endpoint: http://127.0.0.1:8000/train
[data_ingestion_step] Fetching data from API endpoint: http://1

In [12]:
# === Step 2: Missing Data Handling ===
logger.info(" Handling missing data...")
clean_data = missing_value_step(raw_data,strategy='fill',method = 'mode')
logger.info(f"Missing data handled: {clean_data.shape}")

2025-10-22 21:56:09 [INFO] __main__:  Handling missing data...
 Handling missing data...
2025-10-22 21:56:09 [INFO] src.Missing_value_handling: Executing missing value handling strategy.
Executing missing value handling strategy.
2025-10-22 21:56:09 [INFO] src.Missing_value_handling: Filling value with mode strategy
Filling value with mode strategy
2025-10-22 21:56:10 [INFO] src.Missing_value_handling: Missing values filled.
Missing values filled.
2025-10-22 21:56:10 [INFO] __main__: Missing data handled: (40000, 41)
Missing data handled: (40000, 41)


In [15]:
# === Step 3: Sampling (if needed) ===
logger.info("Performing data sampling...")
sampled_data = data_sampling_step(method='smoteenn',df = clean_data,target_col='Target')
logger.info(f"Sampling completed: {sampled_data.shape}")


2025-10-22 22:02:21 [INFO] __main__: Performing data sampling...
Performing data sampling...
2025-10-22 22:02:21 [INFO] src.Data_Samplier: Applying SMOTEENN hybrid resampling. Input size: (40000, 41)
Applying SMOTEENN hybrid resampling. Input size: (40000, 41)
2025-10-22 22:02:26 [INFO] src.Data_Samplier: SMOTEENN complete. Output size: (73091, 41)
SMOTEENN complete. Output size: (73091, 41)
2025-10-22 22:02:26 [INFO] src.Data_Samplier: y Class: Target
1.0    37613
0.0    35478
Name: count, dtype: int64
y Class: Target
1.0    37613
0.0    35478
Name: count, dtype: int64
2025-10-22 22:02:26 [INFO] steps.Data_Sampling_Steps: Applying sampling. using smoteenn
Applying sampling. using smoteenn
2025-10-22 22:02:26 [INFO] __main__: Sampling completed: (73091, 41)
Sampling completed: (73091, 41)


In [ ]:
# === Step 4: Data Split ===
logger.info(" Splitting data into train/test sets...")
X_train, X_test, y_train, y_test = data_split_step(df = sampled_data,target_col = 'Target',test_size = 0.2)
logger.info(f"Split done: Train={len(X_train)}, Test={len(X_test)}")

2025-10-22 22:17:00 [INFO] __main__:  Splitting data into train/test sets...
 Splitting data into train/test sets...
Running single step pipeline to execute step data_split_step
Using an external artifact as step input currently invalidates caching for the step and all downstream steps. Future releases will introduce hashing of artifacts which will improve this behavior.
Initiating a new run for the pipeline: data_split_step.
Uploading external artifact to 'external_artifacts/external_5514d922-a640-40a3-8bc9-1276465e2f8a'.
Finished uploading external artifact 1cce898e-53a7-4417-9706-b764695aaba2.
Caching is disabled by default for data_split_step.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step data_split_step has started.
Step data_split_step has finished in 3.495s.
Pipeline run has finished in 3.733s.
2025-10-22 22:17

In [19]:
# === Step 5: Model Selection ===
logger.info("🤖 Selecting the best model...")
best_model_name, best_model = model_selection_step(X_train,y_train)
logger.info(f"Best model: {best_model_name}")


2025-10-22 23:01:47 [INFO] __main__: 🤖 Selecting the best model...
🤖 Selecting the best model...
Running single step pipeline to execute step model_selection_step
Using an external artifact as step input currently invalidates caching for the step and all downstream steps. Future releases will introduce hashing of artifacts which will improve this behavior.
Using an external artifact as step input currently invalidates caching for the step and all downstream steps. Future releases will introduce hashing of artifacts which will improve this behavior.
Initiating a new run for the pipeline: model_selection_step.
Uploading external artifact to 'external_artifacts/external_98ae98ec-14dc-4bd1-a972-85cc5d3ede52'.
Finished uploading external artifact ec44fa61-d82b-4364-bf34-ff0c11af3fcc.
Uploading external artifact to 'external_artifacts/external_2c40aada-2f50-48e3-a1c0-e4ed86484527'.
Finished uploading external artifact eba28859-f59e-44f5-b08c-b7b4400b7c38.
Caching is disabled by default for m

In [ ]:
#Model Tunning
from src.Model_Tuner import OptunaTuning

#Step 1 Define Tuning configuration
tuner = OptunaTuning(config_dir="config", n_trials=50, cv_folds=5)

#Step 2 Run Tuning to get final model, parameter and best score
best_model, best_params, best_score = tuner.tune("xgbclassifier", XGBClassifier, X_train, y_train)


In [ ]:
# === Step 6: Model Tuning ===
logger.info("Tuning best model...")
tuned_model, best_params, best_score = model_tuning_step(best_model_name=best_model_name,best_model = best_model,X_train=X_train,y_train=y_train)
logger.info(f"Model tuning done: {tuned_model.__class__.__name__} - Final score: {best_score:.2f}")


2025-10-22 23:17:03 [INFO] __main__: Tuning best model...
Tuning best model...
Running single step pipeline to execute step model_tuning_step
Using an external artifact as step input currently invalidates caching for the step and all downstream steps. Future releases will introduce hashing of artifacts which will improve this behavior.
Using an external artifact as step input currently invalidates caching for the step and all downstream steps. Future releases will introduce hashing of artifacts which will improve this behavior.
Using an external artifact as step input currently invalidates caching for the step and all downstream steps. Future releases will introduce hashing of artifacts which will improve this behavior.
Initiating a new run for the pipeline: model_tuning_step.
Uploading external artifact to 'external_artifacts/external_df8fb76f-4b93-40fb-bf32-a8a42df43d7a'.
Finished uploading external artifact dade3a97-34e5-489b-a8be-bdca707fea09.
Uploading external artifact to 'extern

[model_tuning_step] [I 2025-10-22 23:17:06,295] A new study created in memory with name: xgbclassifier_optuna_tuning


  0%|          | 0/50 [00:00<?, ?it/s]

[model_tuning_step] [I 2025-10-22 23:17:11,984] Trial 0 finished with value: 0.984499892403494 and parameters: {'max_depth': 10, 'learning_rate': 0.1745205335405329, 'n_estimators': 1425, 'subsample': 0.6188338544440873, 'colsample_bytree': 0.9549710323520944, 'gamma': 0.18462668954740136, 'min_child_weight': 8, 'reg_alpha': 0.5411819568860157, 'reg_lambda': 0.234058363334949}. Best is trial 0 with value: 0.984499892403494.
[model_tuning_step] [I 2025-10-22 23:17:13,868] Trial 1 finished with value: 0.9585875724452763 and parameters: {'max_depth': 3, 'learning_rate': 0.29087979044703577, 'n_estimators': 448, 'subsample': 0.9941911803922239, 'colsample_bytree': 0.7841158667666361, 'gamma': 0.17559777740408233, 'min_child_weight': 4, 'reg_alpha': 0.9232564061243445, 'reg_lambda': 0.9255221629903605}. Best is trial 0 with value: 0.984499892403494.
[model_tuning_step] [I 2025-10-22 23:17:17,664] Trial 2 finished with value: 0.9583598901523706 and parameters: {'max_depth': 11, 'learning_rat

In [23]:
artifacts_dir = os.path.join(os.getcwd(), "artifacts")
os.makedirs(artifacts_dir, exist_ok=True)


In [24]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_path = os.path.join(artifacts_dir, f"best_model_{timestamp}.pkl")
joblib.dump(tuned_model, model_path)
logger.info(f"💾 Model saved at {model_path}")

2025-10-22 23:23:21 [INFO] __main__: 💾 Model saved at /Users/hanli/cost_ml_202509/Cost_Minimasation_ML/artifacts/best_model_20251022_232321.pkl
💾 Model saved at /Users/hanli/cost_ml_202509/Cost_Minimasation_ML/artifacts/best_model_20251022_232321.pkl
